# How to generate Livetime and Exposure Map from weekly all-sky LAT data

<b> Outline: </b> In this notebook, we show how to generate count and exposure from weekly all-sky LAT data. Here, we use the same data selection which is chosen to match the selection used for [arXiv:2002.01229]. The specification applied for the LAT data is as follows:

- time period: August 4, 2008 (week 9) to June 7, 2018 (week 522),
- event class: SOURCE (P8R2),
- event type: FRONT+BACK,
- energy range: 500 MeV to 500 GeV (24 logarithmically spaced energy bins),
- region of interest: all sky ($0.1^{\circ}\times0.1^{\circ}$ pixels in plate carrée (CAR) projection),
- quality cuts: (DATA_QUAL>0) && (LAT_CONFIG==1),
- zenith angle cut: $>90^{\circ}$.

<b> Note: </b> Before running the notebook, we assume that you have already installed the Fermitools from Conda package manager (https://fermi.gsfc.nasa.gov/ssc/data/analysis/software/) and initiated that by </b> <b>`conda activate fermi'.


## Get the Data and Setup the Analysis

In this thread we will use a pregenerated data set which is contained in a tar archive in the *data* directory of the *fermipy-extra* repository.

In [1]:
%matplotlib inline
import fermipy
import os
import sys
import numpy as np
import matplotlib
import multiprocessing as mp
source_path = "./"

print("fermipy:", fermipy.__version__)
print("numpy:", np.__version__)
print("matplotlib", matplotlib.__version__)

fermipy: v1.0.1
numpy: 1.20.3
matplotlib 3.5.0


## Download the weekly LAT files (Later we would take the data from VRE datalake)

The weekly LAT data files are available from the FSSC [FTP Server](https://heasarc.gsfc.nasa.gov/FTP/fermi/data/lat/). These files contain the LAT data acquired during each week of Fermi's science mission. The file for the current mission week will also be listed, even if the week may not complete.

You can download the LAT weekly files one at a time from the FTP site, or you can use `wget` to download a full set of files.

Once you have the current set of files, you can use the same `wget` command (from the same directory) to download only the newest files. In the event that the LAT team reprocesses data for the full mission, the filenames will change, and `wget` will download a full set of the new, reprocessed, data.

</b> Run this command to download the `p8r2' weekly photon files:

>**Note**: This may take a while to download the data.

In [ ]:
!wget -m -P . -nH --cut-dirs=4 -np -e robots=off https://heasarc.gsfc.nasa.gov/FTP/fermi/data/lat/weekly/p8r2/photon/

In order to do analysis for the full mission, you will also need the mission-long spacecraft file. This file contains entries at 30-second intervals during all periods of active data-taking by the LAT instrument.

</b> Run this command to download the mission spacecraft file:

In [ ]:
!wget -m -P . -nH --cut-dirs=4 -np -e robots=off https://heasarc.gsfc.nasa.gov/FTP/fermi/data/lat/mission/spacecraft/

# Combine the data files

You should now have a full set of weekly files and a mission-long spacecraft file. The next step is to combine the weekly files into a single file. We will use the [gtselect](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/help/gtselect.txt) tool which can also be used for filtering data. But we want to combine the full data set without removing any events, so we will use slightly different parameters.

To combine files, we first create a file list:

In [7]:
!mkdir ./misc
!ls ./weekly/p8r2/photon/lat_photon_weekly* > ./misc/events.txt

mkdir: cannot create directory ‘./data’: File exists


In [8]:
!cat ./misc/events.txt

./weekly/photon/lat_photon_weekly_w009_p302_v001.fits
./weekly/photon/lat_photon_weekly_w010_p302_v001.fits
./weekly/photon/lat_photon_weekly_w011_p302_v001.fits
./weekly/photon/lat_photon_weekly_w012_p302_v001.fits
./weekly/photon/lat_photon_weekly_w013_p302_v001.fits
./weekly/photon/lat_photon_weekly_w014_p302_v001.fits
./weekly/photon/lat_photon_weekly_w015_p302_v001.fits
./weekly/photon/lat_photon_weekly_w016_p302_v001.fits
./weekly/photon/lat_photon_weekly_w017_p302_v001.fits
./weekly/photon/lat_photon_weekly_w018_p302_v001.fits
./weekly/photon/lat_photon_weekly_w019_p302_v001.fits
./weekly/photon/lat_photon_weekly_w020_p302_v001.fits
./weekly/photon/lat_photon_weekly_w021_p302_v001.fits
./weekly/photon/lat_photon_weekly_w022_p302_v001.fits
./weekly/photon/lat_photon_weekly_w023_p302_v001.fits
./weekly/photon/lat_photon_weekly_w024_p302_v001.fits
./weekly/photon/lat_photon_weekly_w025_p302_v001.fits
./weekly/photon/lat_photon_weekly_w026_p302_v001.fits
./weekly/photon/lat_photon_w

If you look at filelist.txt, you will see it is simply a list of all the files in the directory that start with the string `lat_photon_weekly`. 

LAT data has been classified based on the quality of the event reconstruction. The classification can then be used to reduce background and improve performance by filtering out poorly reconstructed events.

With the advent of Pass 8 data reconstruction, there are a myriad of various event classes that are tailored for specific types of analysis. The user should see the [Cicerone](https://fermi.gsfc.nasa.gov/ssc/data/analysis/documentation/Cicerone/Cicerone_Data_Exploration/Data_preparation.html) pages for a full description of the various event classes (and types) available in Pass 8.

For most analyses, the user will be interested in `SOURCE` class photons (`evclass=128`). However, analyses that are intrinsically signal limited require very low background. For example, a user studying large scale, diffuse structures may be interested in using the `CLEAN` (`evclass=256`) or even `ULTRACLEAN` (`evclass=512`) classes.

There are two types of weekly files available; photon files and extended files. The photon weekly files contain all the SOURCE class and are usable for most analyses. The extended files contain the same events as the photon files, but also contain additional reconstruction information about each photon that may be useful if you are trying to characterize the quality of a particular signal, such as for dark matter line searches.

>**NOTE**: For analysis of short timescale events (for example, GRBs) where the flux of the event is much brighter than the accrued background, the user may be interested in using the `TRANSIENT010` (`evclass=64`) event class. If you are interested in investigating transient events within LAT data, you cannot use the weekly files, as they have been prefiltered to contain SOURCE class photons. There are no weekly files that contain the `TRANSIENT` class events.
>
>In order to generate an all-sky dataset of `TRANSIENT` class events, you will need to perform a grid of [data server](https://fermi.gsfc.nasa.gov/cgi-bin/ssc/LAT/LATDataQuery.cgi) queries with the maximum radius of 60 degrees. Once you have downloaded data from the full sky, you will need to combine the files and filter out duplicate events. We recommend using the [FTOOLS](https://heasarc.gsfc.nasa.gov/lheasoft/ftools/ftools_menu.html) suite to perform these tasks.


In [9]:
!punlearn gtselect

Now you're ready to run [gtselect](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/help/gtselect.txt) to combine the data files. Here we will use `INDEF` so that gtselect will not make a selection on event class or event type. See the **gtselect** [help file](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/help/gtselect.txt) for a description of the evclass and evtype cuts.

In [10]:
%%bash
gtselect evclass=INDEF evtype=INDEF
    @./data/events.txt
    ./data/lat_alldata_500MeV.fits
    0
    0
    180
    INDEF
    INDEF
    500
    500000
    180
#### Parameters:
# Input file or files (if multiple files are in a .txt file,
#        don't forget the @ symbol)
# Output file
# RA for new search center
# Dec or new search center
# Radius of the new search region
# Start time (MET in s)
# End time (MET in s)
# Lower energy limit (MeV)
# Upper energy limit (MeV)
# Maximum zenith angle value (degrees)    

Input FT1 file[]     @./data/events.txt
Output FT1 file[]     ./data/lat_alldata_500MeV.fits
RA for new search center (degrees) (0:360) [INDEF]     0
Dec for new search center (degrees) (-90:90) [INDEF]     0
radius of new search region (degrees) (0:180) [INDEF]     180
start time (MET in s) (0:) [INDEF]     INDEF
end time (MET in s) (0:) [INDEF]     INDEF
lower energy limit (MeV) (0:) [30]     500
upper energy limit (MeV) (0:) [300000]     1000000
maximum zenith angle value (degrees) (0:180) [180]     180
Done.


Now with these merged photon and spacecraft files, we can perform any type of analysis that requires using data from many regions on the sky.

We should first filter the data for the proper event class and to remove the earth limb contamination.

We will use the SOURCE class (evclass=128) as it has been tuned to balance statistics with background for long-duration point source analysis.

Here we also include the evtype=3 cut to use both front and back converting event types. We will remove contamination by limiting the reconstructed zenith angle to events at an angle of 90° or less. We will also improve the PSF by excluding events with reconstructed energies below 500 MeV.

Run gtselect to filter the data and apply a zenith cut.

In [11]:
%%bash
gtselect evclass=128 evtype=3
    ./data/lat_alldata_500MeV.fits
    ./data/photons_500MeV_P8R2_SOURCE_V6_w9to522_evtype3.fits
    0
    0
    180
    INDEF
    INDEF
    500
    500000
    90
    
#### Parameters:
# Input file or files (if multiple files are in a .txt file,
#        don't forget the @ symbol)
# Output file
# RA for new search center
# Dec or new search center
# Radius of the new search region
# Start time (MET in s)
# End time (MET in s)
# Lower energy limit (MeV)
# Upper energy limit (MeV)
# Maximum zenith angle value (degrees)

Input FT1 file[@./data/events.txt]     ./data/lat_alldata_500MeV.fits
Output FT1 file[./data/lat_alldata_500MeV.fits]     ./data/photons_500MeV_P8R2_SOURCE_V6_w9to522_evtype3.fits
RA for new search center (degrees) (0:360) [0]     0
Dec for new search center (degrees) (-90:90) [0]     0
radius of new search region (degrees) (0:180) [180]     180
start time (MET in s) (0:) [INDEF]     INDEF
end time (MET in s) (0:) [INDEF]     INDEF
lower energy limit (MeV) (0:) [500]     500
upper energy limit (MeV) (0:) [1000000]     1000000
maximum zenith angle value (degrees) (0:180) [180]     90
Done.


Next, correct the exposure for the events you filtered out.

The way the Fermitools account for exposure is computed based on the Good Time Intervals (GTIs) recorded in the event file. Because we have eliminated some events, we need to update the GTIs for the filtered file. Even though we have used a zenith cut on the data, we will not correct for it here.

For an all-sky analysis, an ROI-based zenith cut would eliminate the entire dataset. Instead, we will later correct the livetime calculation for the zenith angle cut made with [gtselect](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/help/gtselect.txt).

The [gtmktime tool](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/help/gtmktime.txt) updates the GTI list based on events we eliminated in the previous step. It also provides the ability to filter on any of the parameters stored in the spacecraft pointing history file. We will use the most basic filter recommended by the LAT team.

In [17]:
%%bash
gtmktime
    ./mission/spacecraft/lat_spacecraft_merged.fits
    (DATA_QUAL>0)&&(LAT_CONFIG==1)
    no
    ./data/photons_500MeV_P8R2_SOURCE_V6_w9to522_evtype3.fits
    ./data/photons_500MeV_P8R2_SOURCE_V6_w9to522_evtype3_gti.fits
    
#### Parameters specified above are:
# Spacecraft file
# Filter expression
# Apply ROI-based zenith angle cut
# Event data file
# Output event file name    

Spacecraft data file[./mission/spacecraft/spacecraft.fits]     ./mission/spacecraft/lat_spacecraft_merged1.fits
Filter expression[(DATA_QUAL>0)&&(LAT_CONFIG==1)]     (DATA_QUAL>0)&&(LAT_CONFIG==1)
Apply ROI-based zenith angle cut[no]     no
Event data file[./data/photons_500MeV_P8R2_SOURCE_V6_w9to522_evtype3.fits]     ./data/photons_500MeV_P8R2_SOURCE_V6_w9to522_evtype3.fits
Output event file name[./data/photons_500MeV_P8R2_SOURCE_V6_w9to522_evtype3_gti.fits]     ./data/photons_500MeV_P8R2_SOURCE_V6_w9to522_evtype3_gti1.fits


### 3-D (binned) counts map 

Now you will bin the data in preparation for exposure correction. In order to correct the map for exposure, you have to account for the changing response with energy. To do this, bin the data into a counts cube using [gtbin](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/help/gtbin.txt), but with a single energy bin.

In [18]:
%%bash
gtbin
    CCUBE
    ./data/photons_500MeV_P8R2_SOURCE_V6_w9to522_evtype3_gti.fits
    ./data/CCUBE_P8R2_SOURCE_V6_w9to522_evtype3_nxpix3600_nypix1800_binsz0.1_Elo500_Ehi500000.fits
    ./mission/spacecraft/lat_spacecraft_merged.fits
    3600
    1800
    0.1
    GAL
    0
    0
    0
    CAR
    LOG
    500
    500000
    24
    
#### Parameters:
# Type of output file (CCUBE|CMAP|LC|PHA1|PHA2)
# Event data file name
# Output file name
# Spacecraft data file name [NONE is valid]
# Size of the X axis in pixels
# Size of the Y axis in pixels
# Image scale (in degrees/pixel)
# Coordinate system (CEL - celestial, GAL -galactic) (CEL|GAL)
# First coordinate of image center in degrees (RA or galactic l)
# Second coordinate of image center in degrees (DEC or galactic b)
# Rotation angle of image axis, in degrees
# Projection method e.g. AIT|ARC|CAR|GLS|MER|NCP|SIN|STG|TAN
#Algorithm for defining energy bins (FILE|LIN|LOG)
#Start value for first energy bin in MeV
#Stop value for last energy bin in MeV
#Number of logarithmically uniform energy bin

This is gtbin version HEAD
Type of output file (CCUBE|CMAP|LC|PHA1|PHA2|HEALPIX) [CCUBE]     CCUBE
Event data file name[./data/photons_500MeV_P8R2_SOURCE_V6_w9to522_evtype3_gti.fits]     ./data/photons_500MeV_P8R2_SOURCE_V6_w9to522_evtype3_gti1.fits
Output file name[./data/CCUBE_P8R2_SOURCE_V6_w9to522_evtype3_nxpix3600_nypix1800_binsz0.1_Elo500_Ehi500000.fits]     ./data/CCUBE_P8R2_SOURCE_V6_w9to522_evtype3_nxpix3600_nypix1800_binsz0.1_Elo500_Ehi5000001.fits
Spacecraft data file name[./mission/spacecraft/spacecraft.fits]     ./mission/spacecraft/lat_spacecraft_merged1.fits
Size of the X axis in pixels[3600]     3600
Size of the Y axis in pixels[1800]     1800
Image scale (in degrees/pixel)[0.1]     0.1
Coordinate system (CEL - celestial, GAL -galactic) (CEL|GAL) [GAL]     GAL
First coordinate of image center in degrees (RA or galactic l)[0]     0
Second coordinate of image center in degrees (DEC or galactic b)[0]     0
Rotation angle of image axis, in degrees[0]     0
Projection method

### Calculate Exposure Maps

Because Fermi data are sparse, the accumulated exposure time for any given point in the sky is based on the observatory's pointing history and the response of the instrument to events at various incidence angles and different energies.

The LAT team has calculated how the instrument responds by modeling the entire instrument and then running a MonteCarlo simulation of many millions of events of different energies and directions. These simulations characterize how the gamma rays propagate through an ideal instrument, and the information is recorded in the Instrument Response Functions (IRF). Since the response depends on how much data filtering is performed, there are different IRFs for the different event classes.

To see which IRFs are available from within the Fermitools, run the [gtirfs](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/help/gtirfs.txt) command.

In [3]:
!gtirfs

P7CLEAN_V6 ( = P7CLEAN_V6::BACK + P7CLEAN_V6::FRONT )
P7CLEAN_V6::BACK
P7CLEAN_V6::FRONT
P7REP_CLEAN_V10 ( = P7REP_CLEAN_V10::BACK + P7REP_CLEAN_V10::FRONT )
P7REP_CLEAN_V10::BACK
P7REP_CLEAN_V10::FRONT
P7REP_CLEAN_V15 ( = P7REP_CLEAN_V15::BACK + P7REP_CLEAN_V15::FRONT )
P7REP_CLEAN_V15::BACK
P7REP_CLEAN_V15::FRONT
P7REP_SOURCE_V10 ( = P7REP_SOURCE_V10::BACK + P7REP_SOURCE_V10::FRONT )
P7REP_SOURCE_V10::BACK
P7REP_SOURCE_V10::FRONT
P7REP_SOURCE_V15 ( = P7REP_SOURCE_V15::BACK + P7REP_SOURCE_V15::FRONT )
P7REP_SOURCE_V15::BACK
P7REP_SOURCE_V15::FRONT
P7REP_TRANSIENT_V10 ( = P7REP_TRANSIENT_V10::BACK + P7REP_TRANSIENT_V10::FRONT )
P7REP_TRANSIENT_V10::BACK
P7REP_TRANSIENT_V10::FRONT
P7REP_TRANSIENT_V15 ( = P7REP_TRANSIENT_V15::BACK + P7REP_TRANSIENT_V15::FRONT )
P7REP_TRANSIENT_V15::BACK
P7REP_TRANSIENT_V15::FRONT
P7REP_ULTRACLEAN_V10 ( = P7REP_ULTRACLEAN_V10::BACK + P7REP_ULTRACLEAN_V10::FRONT )
P7REP_ULTRACLEAN_V10::BACK
P7REP_ULTRACLEAN_V10::FRONT
P7REP_ULTRACLEAN_V15 ( = P7REP_ULTRACL

### Calculate the Livetime

For this notebbok, we have used the IRF `P8R2_SOURCE_V6`.

Calculating exposure for an all-sky map first requires calculating the instrument livetime for the entire sky (using [gtltcube](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/help/gtltcube.txt)) and then convolving the livetime with the IRF (using [gtexpmap2](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/help/gtexpmap.txt)).

The pixel size for the livetime does not need to match the binned data, but you do need to provide the event file as [gtltcube](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/help/gtltcube.txt) uses the good time intervals to calculate the livetime at each position.

You will also need to correct the livetime for the zenith angle cut made with [gtselect](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/help/gtselect.txt) earlier. To do this, use the `zmax` option on the command line and match the value used with [gtselect](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/help/gtselect.txt). This modifies the livetime calculation to account for the events you removed earlier.
Here is an example of the input for [gtltcube](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/help/gtltcube.txt):

In [19]:
 %%bash
gtltcube zmax=90
    ./data/photons_500MeV_P8R2_SOURCE_V6_w9to522_evtype3_gti.fits
    ./mission/spacecraft/lat_spacecraft_merged.fits
    ./data/expCube_500MeV_P8R2_SOURCE_V6_w9to522_evtype3.fits
    0.025
    1  
    
#### gtltcube Parameters:
# Event data file
# Spacecraft data file
# Output file
# Step size in cos(theta) (0.:1.)
# Pixel size (degrees)
#
# May take a while to finish    

Event data file[./data/photons_500MeV_P8R2_SOURCE_V6_w9to522_evtype3_gti.fits]    ./data/photons_500MeV_P8R2_SOURCE_V6_w9to522_evtype3_gti1.fits
Spacecraft data file[./mission/spacecraft/spacecraft.fits]    ./mission/spacecraft/lat_spacecraft_merged1.fits
Output file[./data/expCube_500MeV_P8R2_SOURCE_V6_w9to522_evtype3.fits]    ./data/expCube_500MeV_P8R2_SOURCE_V6_w9to522_evtype31.fits
Step size in cos(theta) (0.:1.) [0.025]    0.025
Pixel size (degrees)[1]    1  


Working on file ./mission/spacecraft/lat_spacecraft_merged1.fits
.....................!


### Generate an Exposure Cube

Now you can calculate the exposure at each point in the sky in each energy bin (we used only one bin) using [gtexpcube2](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/help/gtexpcube2.txt).

You will need to enter the pixel geometry and energy binning to match what you used in the counts cube.

Also, be sure to use `CENTER` for the energy layer reference. If you use `EDGE`, you will get an additional energy plane in the output file that causes complications later.

In [20]:
%%bash
gtexpcube2 
    ./data/expCube_500MeV_P8R2_SOURCE_V6_w9to522_evtype3.fits
    none
    ./data/expmap_CCUBE_P8R2_SOURCE_V6_w9to522_evtype3_nxpix3600_nypix1800_binsz0.1_Elo500_Ehi500000.fits
    P8R2_SOURCE_V6
    3600
    1800
    0.1
    0
    0
    0
    AIT
    GAL
    500
    500000
    1   

#### gtexpcube2 Parameters:
# Livetime cube data file
# Counts map data file [NONE is valid]
# Output file name
# Response functions
# Size of the X axis in pixels
# Size of the Y axis in pixels
# Image scale (in degrees/pixel)
# Coordinate system (CEL - celestial, GAL -galactic) (CEL|GAL)
# First coordinate of image center in degrees (RA or galactic l)
# Second coordinate of image center in degrees (DEC or galactic b)
# Rotation angle of image axis, in degrees
# Projection method e.g. AIT|ARC|CAR|GLS|MER|NCP|SIN|STG|TAN
#Start energy of first bin in MeV
#Stop energy of last bin in MeV
#Number of logarithmically-spaced energy bins

# This will generate an exposure cube.
# This may take a long time. 

Livetime cube file[./data/expCube_500MeV_P8R2_SOURCE_V6_w9to522_evtype3.fits]     ./data/expCube_500MeV_P8R2_SOURCE_V6_w9to522_evtype31.fits
Counts map file[none]     none
Output file name[./data/expmap_CCUBE_P8R2_SOURCE_V6_w9to522_evtype3_nxpix3600_nypix1800_binsz0.1_Elo500_Ehi500000.fits]     ./data/expmap_CCUBE_P8R2_SOURCE_V6_w9to522_evtype3_nxpix3600_nypix1800_binsz0.1_Elo500_Ehi5000001.fits
Response functions to use[P8R2_SOURCE_V6]     P8R2_SOURCE_V6
Size of the X axis in pixels[3600]     3600
Size of the Y axis in pixels[1800]     1800
Image scale (in degrees/pixel)[0.1]     0.1
First coordinate of image center in degrees (RA or galactic l)[0]     0
Second coordinate of image center in degrees (DEC or galactic b)[0]     0
Rotation angle of image axis, in degrees[0]     0
Projection method e.g. AIT|ARC|CAR|GLS|MER|NCP|SIN|STG|TAN[AIT]     AIT
Coordinate system (CEL - celestial, GAL -galactic) (CEL|GAL) [GAL]     GAL
Start energy (MeV) of first bin[500]     500
Stop energy (MeV) of

Computing binned exposure map....................!


**Note**: You could use CALDB for the IRF, but if this doesn't work, try explicitly specifying the IRF. 